In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
from multiprocessing import Process, Queue
import sys
import time
import random
import math


max_num = 1000
train_path = '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/'
valid_path = '/HardDisk/Irving/ILSVRC2012/validation/'
label_path = '/HardDisk/Irving/ILSVRC2012/label_data/'
cores = 6   # the number of cores


classes = os.listdir(train_path)


labels_dict = {}
for i in range(len(classes)):
    labels_dict[classes[i]]=i

# make a list which include each picture filename and classname
images_labels_list = []
for i in range(len(classes)):
    path = train_path + classes[i] + '/'
    images_files = os.listdir(path)
    label = str(labels_dict[classes[i]])
    for image_file in images_files:
        images_labels_list.append(path+','+image_file+','+classes[i])
random.shuffle(images_labels_list)


In [2]:
images_labels_list


['/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n04493381/,n04493381_15907.JPEG,n04493381',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n02769748/,n02769748_18564.JPEG,n02769748',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n03657121/,n03657121_30426.JPEG,n03657121',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n04118538/,n04118538_4402.JPEG,n04118538',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n03207743/,n03207743_1455.JPEG,n03207743',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n03956157/,n03956157_28.JPEG,n03956157',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n02117135/,n02117135_7740.JPEG,n02117135',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n04392985/,n04392985_7347.JPEG,n04392985',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n03888257/,n03888257_7610.JPEG,n03888257',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n04557648/,n04557648_9468.JPEG,n04557648',
 '/HardDisk/Irving/ILSVRC2012/ILSVRC2012_img_train/n0399106

In [3]:
valid_classes = []
with open(label_path+'ILSVRC2012_validation_ground_truth.txt','r') as f:
    valid_classes = [line.strip() for line in f.readlines()]

# build valid dataset list including( road, filename, classname)
valid_images_labels_list = []
valid_images_files = os.listdir('/HardDisk/Irving/ILSVRC2012/validation/')
valid_images_files.sort()      # if the order of the image file is not right , the sort function may be help
for file_item in valid_images_files:
    number = int(file_item[15:23])-1        # reason -1 :the valid_classes number from 0
    #print(number)
    valid_images_labels_list.append(valid_path+','+file_item+','+valid_classes[number])
#valid_images_files 

In [4]:
valid_classes

['490',
 '361',
 '171',
 '822',
 '297',
 '482',
 '13',
 '704',
 '599',
 '164',
 '649',
 '11',
 '73',
 '286',
 '554',
 '6',
 '648',
 '399',
 '749',
 '545',
 '13',
 '204',
 '318',
 '693',
 '399',
 '304',
 '102',
 '207',
 '480',
 '780',
 '644',
 '275',
 '14',
 '954',
 '249',
 '790',
 '501',
 '547',
 '809',
 '606',
 '297',
 '927',
 '424',
 '156',
 '60',
 '983',
 '256',
 '207',
 '281',
 '456',
 '413',
 '498',
 '561',
 '750',
 '182',
 '267',
 '118',
 '893',
 '597',
 '840',
 '836',
 '107',
 '647',
 '471',
 '945',
 '451',
 '214',
 '790',
 '291',
 '837',
 '707',
 '193',
 '397',
 '568',
 '401',
 '705',
 '200',
 '202',
 '31',
 '949',
 '361',
 '98',
 '709',
 '483',
 '563',
 '695',
 '122',
 '497',
 '914',
 '476',
 '102',
 '199',
 '104',
 '221',
 '138',
 '257',
 '188',
 '436',
 '229',
 '52',
 '377',
 '200',
 '597',
 '544',
 '131',
 '935',
 '602',
 '342',
 '508',
 '748',
 '617',
 '901',
 '184',
 '513',
 '204',
 '999',
 '782',
 '922',
 '869',
 '598',
 '596',
 '701',
 '970',
 '452',
 '631',
 '817',
 '5

In [5]:
valid_images_labels_list

['/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000001.JPEG,490',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000002.JPEG,361',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000003.JPEG,171',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000004.JPEG,822',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000005.JPEG,297',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000006.JPEG,482',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000007.JPEG,13',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000008.JPEG,704',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000009.JPEG,599',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000010.JPEG,164',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000011.JPEG,649',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000012.JPEG,11',
 '/HardDisk/Irving/ILSVRC2012/validation/,ILSVRC2012_val_00000013.JPEG,73',
 '

In [6]:
# transfrom the image data and labels to format TFRECORD
def make_example(image, height, width, label, bbox, text):
    colorspace = b'RGB'
    channels = 3
    img_format = b'JPEG'
    return tf.train.Example(features=tf.train.Features(feature={
        'image' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
        'height' : tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'width' : tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'channels' : tf.train.Feature(int64_list=tf.train.Int64List(value=[channels])),
        'colorspace' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[colorspace])),
        'img_format' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_format])),
        'label' : tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
        'bbox_xmin' : tf.train.Feature(float_list=tf.train.FloatList(value=bbox[0])),
        'bbox_xmax' : tf.train.Feature(float_list=tf.train.FloatList(value=bbox[2])),
        'bbox_ymin' : tf.train.Feature(float_list=tf.train.FloatList(value=bbox[1])),
        'bbox_ymax' : tf.train.Feature(float_list=tf.train.FloatList(value=bbox[3])),
        'text' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[text]))
    }))

In [7]:
# read bbox file
bbox_list = {}
with open('bbox_train.csv','r') as bboxfile:
    records = bboxfile.readlines()
    for record in records:
        fields = record.strip().split(',')
        filename = fields[0][:-4]
        xmin = [float(x) for x in fields[1].split(';')]
        ymin = [float(x) for x in fields[2].split(';')]
        xmax = [float(x) for x in fields[3].split(';')]
        ymax = [float(x) for x in fields[4].split(';')]
        bbox_list[filename] = [xmin, ymin, xmax, ymax]

In [8]:
valid_text = {}
with open('/home/Irving/AI/Alex/Data/label_valid_train.txt','r') as vtfile:
    records = vtfile.readlines()
    
    for record in records:
        fields = record.strip().split('\t')
        v = fields[0]
        t = fields[1]
        valid_text[v] = t

In [9]:
# read description of labels
labels_text = {}
with open('/home/Irving/AI/Alex/Data/imagenet_metadata.txt', 'r') as metafile:
    records = metafile.readlines()
    for record in records:
        fields = record.strip().split('\t')
        label = fields[0]
        text = fields[1]
        labels_text[label] = text

In [10]:
#labels_text

In [15]:
# 这个函数用来生成TFRECORD文件，第一个参数是列表，每个元素是图片文件名加类名，第二个参数是写入的目录名
# 第三个参数是文件名的起始序号，第四个参数是队列名称，用于和父进程发送消息
def gen_tfrecord(trainrecords, targetfolder, startnum, queue):
    tfrecords_file_num = startnum
    file_num = 0
    total_num = len(trainrecords)
    pid = os.getpid()
    queue.put((pid, file_num))
    writer = tf.python_io.TFRecordWriter(targetfolder + "train_" + str(tfrecords_file_num) + ".tfrecord")
    for record in trainrecords:
        file_num += 1
        fields = record.split(',')
        img = cv2.imread(fields[0] + fields[1])
        height, width, _ = img.shape
        img_jpg = cv2.imencode('.jpg', img)[1].tobytes()
        label = labels_dict[fields[2]]
        bbox = []
        try:
            bbox = bbox_list[fields[1][:-5]]
        except KeyError:
            bbox = [[], [], [], []]
        text = labels_text[fields[2]]
        print("###")
        print(fields[2])
        print(text)
        print("###")
        ex = make_example(img_jpg, height, width, label, bbox, text.encode())
        writer.write(ex.SerializeToString())
        # 每写入100条记录，向父进程发送消息，报告进度
        if file_num % 100 == 0:
            queue.put((pid, file_num))
        if file_num % max_num == 0:
            writer.close()
            tfrecords_file_num += 1
            writer = tf.python_io.TFRecordWriter(targetfolder + "train_" + str(tfrecords_file_num) + ".tfrecord")
    writer.close()
    queue.put((pid, file_num))


# 这个函数用来多进程生成TFRECORD文件，第一个参数是要处理的图片的文件名列表，第二个参数是需要用的CPU核心数
# 第三个参数写入的文件目录名
def process_in_queues(fileslist, cores, targetfolder):
    total_files_num = len(fileslist)
    each_process_files_num = int(total_files_num / cores)
    files_for_process_list = []
    #print(files_number_list)
    for i in range(cores - 1):
        files_for_process_list.append(fileslist[i * each_process_files_num:(i + 1) * each_process_files_num])
    files_for_process_list.append(fileslist[(cores - 1) * each_process_files_num:])
    #print(1)
    files_number_list = [len(l) for l in files_for_process_list]
    #print(2)


    each_process_tffiles_num = math.ceil(each_process_files_num / max_num)

    queues_list = []
    processes_list = []
    #print(3)
    for i in range(cores):
        queues_list.append(Queue())
        # queue = Queue()
        processes_list.append(Process(target=gen_tfrecord,
                                      args=(files_for_process_list[i], targetfolder,
                                            each_process_tffiles_num * i + 1, queues_list[i],)))

    for p in processes_list:
        Process.start(p)

    # 父进程循环查询队列的消息，并且每0.5秒更新一次
    while (True):
        try:
            total = 0
            progress_str = ''
            #print(4)
            for i in range(cores):
                msg = queues_list[i].get()
                total += msg[1]
                progress_str += 'PID' + str(msg[0]) + ':' + str(msg[1]) + '/' + str(files_number_list[i]) + '|'
            
            progress_str += '\r'
            print(progress_str, end='')
            if total == total_files_num:
                for p in processes_list:
                    p.terminate()
                    p.join()
                break
            time.sleep(0.5)
        except:
            break
    return total




In [16]:
if __name__ == '__main__':
    
    print('Start processing train data using %i CPU cores:'%cores)
    starttime=time.time()         
    total_processed = process_in_queues(images_labels_list, cores, targetfolder='/home/Irving/AI/Alex/Data/train_tf123/')
    endtime=time.time()
    print('\nProcess finish, total process %i images in %i seconds'%(total_processed, int(endtime-starttime)))




Start processing train data using 6 CPU cores:
###
n04493381
tub, vat
###
###
n04086273
revolver, six-gun, six-shooter
###
###
n07584110
consomme
###
###
n03983396
pop bottle, soda bottle
###
###
n04522168
vase
###
###
n03483316
hand blower, blow dryer, blow drier, hair dryer, hair drier
###
###
n02892767
brassiere, bra, bandeau
###
###17274:0/213527|PID17276:0/213527|PID17278:0/213527|PID17280:0/213527|PID17284:0/213527|PID17292:0/213532|
n03216828
dock, dockage, docking facility
###
###
n03595614
jersey, T-shirt, tee shirt
###
###
n01985128
crayfish, crawfish, crawdad, crawdaddy
###
###
n02769748
backpack, back pack, knapsack, packsack, rucksack, haversack
###
###
n01692333
Gila monster, Heloderma suspectum
###
###
n04116512
rubber eraser, rubber, pencil eraser
###
###
n02105251
briard
###
###
n01770393
scorpion
###
###
n02169497
leaf beetle, chrysomelid
###
###
n04332243
strainer
###
###
n07697537
hotdog, hot dog, red hot
###
###
n02092002
Scottish deerhound, deerhound
###
###
n0209

###
###
n02108551
Tibetan mastiff
###
###
n02106166
Border collie
###
###
n02093256
Staffordshire bullterrier, Staffordshire bull terrier
###
###
n02268443
dragonfly, darning needle, devil's darning needle, sewing needle, snake feeder, snake doctor, mosquito hawk, skeeter hawk
###
###
n02692877
airship, dirigible
###
###
n02777292
balance beam, beam
###
###
###
n02790996
n04136333
barbell
sarong
###
###
###
n01943899
conch
###
###
n01629819
European fire salamander, Salamandra salamandra
###
###
n12620546
hip, rose hip, rosehip
###
###
n04399382
teddy, teddy bear
###
###
n02840245
binder, ring-binder
###
###
###
n02823750
n03770679
beer glass
minivan
###
###
###
n02389026
sorrel
###
###
n02106382
Bouvier des Flandres, Bouviers des Flandres
###
###
n01687978
agama
###
###
n02009912
American egret, great white heron, Egretta albus
###
###
n04266014
space shuttle
###
###
n07584110
consomme
###
###
n03535780
horizontal bar, high bar
###
###
n03085013
computer keyboard, keypad
###
###
n0377

n04536866
###
violin, fiddle
###
###
n03444034
go-kart
###
###
n02492035
capuchin, ringtail, Cebus capucinus
###
###
n03208938
disk brake, disc brake
###
###
n03920288
Petri dish
###
###
n04517823
vacuum, vacuum cleaner
###
###
n03903868
pedestal, plinth, footstall
###
###
n01751748
sea snake
###
###
n03000247
chain mail, ring mail, mail, chain armor, chain armour, ring armor, ring armour
###
###
n02165105
tiger beetle
###
###
n04552348
warplane, military plane
###
###
n04409515
tennis ball
###
###
n03710721
maillot, tank suit
###
###
n04398044
teapot
###
###
n04371430
swimming trunks, bathing trunks
###
###
n04153751
screw
###
###
n03476684
hair slide
###
###
n02089973
English foxhound
###
###
n02106662
German shepherd, German shepherd dog, German police dog, alsatian
###
###
n02917067
bullet train, bullet
###
###
n04501370
turnstile
###
###
n02804414
bassinet
###
###
n07583066
guacamole
###
###
n03594734
jean, blue jean, denim
###
###
n01843383
toucan
###
###
n01664065
loggerhead, lo

Process Process-14:
Traceback (most recent call last):
  File "/home/Irving/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()



Process finish, total process 0 images in 13 seconds


  File "/home/Irving/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-15-e22357755cb6>", line 13, in gen_tfrecord
    img = cv2.imread(fields[0] + fields[1])
KeyboardInterrupt
Process Process-13:
Traceback (most recent call last):
  File "/home/Irving/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/Irving/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-15-e22357755cb6>", line 13, in gen_tfrecord
    img = cv2.imread(fields[0] + fields[1])
KeyboardInterrupt
Process Process-16:
Traceback (most recent call last):
  File "/home/Irving/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/Irving/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs

In [13]:
def gen_tfrecord_valid(trainrecords, targetfolder, startnum, queue):
    tfrecords_file_num = startnum
    file_num = 0
    total_num = len(trainrecords)
    pid = os.getpid()
    queue.put((pid, file_num))
    writer = tf.python_io.TFRecordWriter(targetfolder + "valid_" + str(tfrecords_file_num) + ".tfrecord")
    for record in trainrecords:
        file_num += 1
        fields = record.split(',')
        
        
        
        #print(fields)
        #print(fields[2])
        
        
        
        img = cv2.imread(fields[0] + fields[1])
        height, width, _ = img.shape
        img_jpg = cv2.imencode('.jpg', img)[1].tobytes()
        
        label = int(fields[2])
        
        bbox = []
        try:
            bbox = bbox_list[fields[1][:-5]]
        except KeyError:
            bbox = [[], [], [], []]
        
        
        
        train_text = valid_text[fields[2]]   
        text = labels_text[train_text]
        
        
        
        
        ex = make_example(img_jpg, height, width, label, bbox, text.encode())
        writer.write(ex.SerializeToString())
        # 每写入100条记录，向父进程发送消息，报告进度
        if file_num % 100 == 0:
            queue.put((pid, file_num))
        if file_num % max_num == 0:
            writer.close()
            tfrecords_file_num += 1
            writer = tf.python_io.TFRecordWriter(targetfolder + "valid_" + str(tfrecords_file_num) + ".tfrecord")
    writer.close()
    queue.put((pid, file_num))
    
    
# 这个函数用来多进程生成TFRECORD文件，第一个参数是要处理的图片的文件名列表，第二个参数是需要用的CPU核心数
# 第三个参数写入的文件目录名
def process_in_queues_valid(fileslist, cores, targetfolder):
    total_files_num = len(fileslist)
    each_process_files_num = int(total_files_num / cores)
    files_for_process_list = []
    #print(files_number_list)
    for i in range(cores - 1):
        files_for_process_list.append(fileslist[i * each_process_files_num:(i + 1) * each_process_files_num])
    files_for_process_list.append(fileslist[(cores - 1) * each_process_files_num:])
    #print(1)
    files_number_list = [len(l) for l in files_for_process_list]
    #print(2)


    each_process_tffiles_num = math.ceil(each_process_files_num / max_num)

    queues_list = []
    processes_list = []
    #print(3)
    for i in range(cores):
        queues_list.append(Queue())
        # queue = Queue()
        processes_list.append(Process(target=gen_tfrecord_valid,
                                      args=(files_for_process_list[i], targetfolder,
                                            each_process_tffiles_num * i + 1, queues_list[i],)))

    for p in processes_list:
        Process.start(p)

    # 父进程循环查询队列的消息，并且每0.5秒更新一次
    while (True):
        try:
            total = 0
            progress_str = ''
            #print(4)
            for i in range(cores):
                msg = queues_list[i].get()
                total += msg[1]
                progress_str += 'PID' + str(msg[0]) + ':' + str(msg[1]) + '/' + str(files_number_list[i]) + '|'
            
            progress_str += '\r'
            print(progress_str, end='')
            if total == total_files_num:
                for p in processes_list:
                    p.terminate()
                    p.join()
                break
            time.sleep(0.5)
        except:
            break
    return total

In [14]:

    
if __name__ == '__main__':
    print('Start processing validation data using %i CPU cores:' % cores)
    starttime = time.time()
    total_processed = process_in_queues_valid(valid_images_labels_list, cores, targetfolder='/home/Irving/AI/Alex/Data/valid_tf/')
    endtime = time.time()
    #print('\nProcess finish, total process %i images, using %i seconds' % (total_processed, int(endtime - starttime)))
    print('\nProcess finish, total process %i images in %i seconds'%(total_processed, int(endtime-starttime)))

Start processing validation data using 6 CPU cores:
PID31287:8333/8333|PID31289:8333/8333|PID31291:8333/8333|PID31293:8333/8333|PID31295:8333/8333|PID31297:8335/8335|
Process finish, total process 50000 images in 682 seconds
